In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

- event_data - данные о действиях, которые совершают студенты:

    - step_id - id стэпа
    - timestamp - время наступления события в формате unix date
    - user_id - id юзера
    - action - события:
    
        - viewed - пользователь впервые увидел задание/урок
        - passed - пользователь успешно сдал задание
        - discovered - пользователь перешел на стэп
        - started_attempt - пользователь начал решать задание
        

In [2]:
event_data_train = pd.read_csv("datas/event_data_train.zip")
event_data_train.shape


(3480703, 4)

In [3]:
event_data_train.head()

,step_id,timestamp,action,user_id
0,32815,1434340848,viewed,17632
1,32815,1434340848,passed,17632
2,32815,1434340848,discovered,17632
3,32811,1434340895,discovered,17632
4,32811,1434340895,viewed,17632


In [4]:
event_data_test = pd.read_csv("datas/events_data_test.zip")
event_data_test.shape

(274768, 4)

In [5]:
event_data_test.head()

,step_id,timestamp,action,user_id
0,30456,1526893787,viewed,24417
1,30456,1526893797,viewed,24417
2,30456,1526893954,viewed,24417
3,30456,1526895780,viewed,24417
4,30456,1526893787,discovered,24417


- submission_data - данные о времени и статусах сабмитов к практическим заданиям:

    - step_id - id стэпа
    - timestamp - время отправки решения в формате unix date
    - user_id - анонимизированный id юзера
    - submission_status - статус решения:

        - correct - верное решение
        - wrong - неверное решение
        

In [6]:
submission_data_train = pd.read_csv("datas/submissions_data_train.zip")
submission_data_train.shape

(509104, 4)

In [7]:
submission_data_train.head()

,step_id,timestamp,submission_status,user_id
0,31971,1434349275,correct,15853
1,31972,1434348300,correct,15853
2,31972,1478852149,wrong,15853
3,31972,1478852164,correct,15853
4,31976,1434348123,wrong,15853


In [8]:
submission_data_test = pd.read_csv("datas/submission_data_test.zip")
submission_data_test.shape

(42261, 4)

In [9]:
submission_data_test.head()

,step_id,timestamp,submission_status,user_id
0,31971,1526800961,wrong,24370
1,31971,1526800976,wrong,24370
2,31971,1526800993,wrong,24370
3,31971,1526801054,correct,24370
4,31972,1526800664,wrong,24370


- train-данные содержат данные почти за 3 года по всем пользователей без какой-либо фильтрации
- test-данные содержат данные о действиях каждом пользователе за __два первых дня__ конкретного пользователя

Напишем функции для предобработки данных:

In [10]:
def time_filter(data, days=2):
    """
    Выбирает для переданного датасета все события для каждого пользователя,
    укладывающиеся в интервал двух первых дней.
    """
    time_threshold = 60 * 60 * 24 * days

    user_min_time = data.groupby('user_id').agg({'timestamp': 'min'}) \
                        .rename(columns={'timestamp': 'min_timestamp'}) \
                        .reset_index()

    data_filtered = pd.merge(data, user_min_time, on='user_id', how='outer')

    data_filtered = data_filtered.query(f"timestamp <= min_timestamp + {time_threshold}")

    assert data_filtered['user_id'].nunique() == data['user_id'].nunique()

    return data_filtered.drop(['min_timestamp'], axis=1)

In [11]:
def base_features(event_data, submission_data):
    """
    Создание датасета с подсчётом всех действий каждого пользователя и
    количеством правильных и неправильных ответов.
    """
    user_event = pd.pivot_table(data=event_data, values='step_id',
                                index='user_id', columns='action',
                                aggfunc='count', fill_value=0) \
                                .reset_index().rename_axis('', axis=1)
    
    user_submission = pd.pivot_table(data=submission_data, values='step_id',
                                index='user_id', columns='submission_status',
                                aggfunc='count', fill_value=0) \
                                .reset_index().rename_axis('', axis=1)
    
    user_data = pd.merge(user_event, user_submission, on='user_id', how='outer') \
                  .fillna(0)
    
    assert user_data['user_id'].nunique() == event_data['user_id'].nunique()

    return user_data


In [12]:
def target_col(submission_data, threshold=40):
    """
    Вычисляет целевую переменную для каждого пользователя по таблице
    сабмитов. Принятый порог равен 40 баллам. Если пользователь набрал
    40 баллов - он прошёл курс.
    """

    user_correct_count = submission_data[submission_data['submission_status'] == 'correct'] \
                            .groupby('user_id').agg({'step_id': 'count'}) \
                            .rename(columns={'step_id': 'completed_steps'}) \
                            .reset_index()
    
    user_correct_count['passed_course'] = (user_correct_count['completed_steps'] >= threshold) \
                                                .astype('int')
    
    return user_correct_count.drop(['completed_steps'], axis=1)

In [13]:
def steps_tried(submission_data):
    """
    Вычисляет количество степов, которые попытался решить пользователь
    """

    steps_tried_data = submission_data.groupby('user_id')['step_id'].nunique().to_frame() \
                                      .rename(columns={'step_id': 'steps_tried'}) \
                                      .reset_index()

    return steps_tried_data

In [14]:
def correct_ratio(data):
    """
    Вычисление фичи с долей правильных ответов
    """
    data['correct_ratio'] = (data['correct'] / (data['correct'] + data['wrong'])).fillna(0)
    
    return data

In [15]:
def make_train_data(event_data, submission_data):
    """
    Формирует датасеты X и y для тренировки датасета
    """
    # фильтрация двух первых дней на курсе
    event_data_2days = time_filter(event_data)
    submission_data_2days = time_filter(submission_data)
    # базовые фичи
    user_data = base_features(event_data_2days, submission_data_2days)
    # целевая колонка
    user_data_target = target_col(submission_data)
    # количество уникальных заданий, к которым приступил пользователь
    users_steps_tried = steps_tried(submission_data_2days)
    # доля правильных ответов
    user_data = correct_ratio(user_data)
    # присоединим данные
    user_data = user_data.merge(user_data_target, how='outer').fillna(0)
    user_data = user_data.merge(users_steps_tried, how='outer').fillna(0)

    X = user_data.drop(['passed_course'], axis=1)
    y = user_data['passed_course'].map(int)

    return X, y

In [16]:
def make_test_data(event_data, submission_data):
    """
    Формирует датасет y для тестирования датасета
    """
    # базовые фичи
    user_data = base_features(event_data, submission_data)
    # количество уникальных заданий, к которым приступил пользователь
    users_steps_tried = steps_tried(submission_data)
    # доля правильных ответов
    user_data = correct_ratio(user_data)
    # присоединим данные
    user_data = user_data.merge(users_steps_tried, how='outer').fillna(0)

    X = user_data

    return X

Создадим тренировочный датасет.

In [17]:
X_train, y_train = make_train_data(event_data_train, submission_data_train)

Создадим тестовый датасет.

In [18]:
X_test = make_test_data(event_data_test, submission_data_test)

Запишем данные в .csv файлы

In [19]:
X_train.to_csv("datas/X_train.zip", index_label=False)
y_train.to_csv("datas/y_train.zip", index_label=False)
X_test.to_csv("datas/X_test.zip", index_label=False)